## NTV entities are compatible with tabular data tools
- Example with Pandas

In [ ]:
from json_ntv import NtvSingle, NtvList, Ntv, NtvConnector, NtvType, Namespace
from datetime import date, datetime

In [ ]:
field_data = {'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21']}
tab_data   = {'index':           [1, 2, 3],
              'dates::datetime': ['1964-01-01', '1985-02-05', '2022-01-21'], 
              'value':           [10, 20, 30],
              'value32::int32':  [10, 20, 30],
              'coord::point':    [[1,2], [3,4], [5,6]],
              'names::string':   ['john', 'eric', 'judith']}

field = Ntv.obj({':field': field_data})
tab   = Ntv.obj({':tab'  : tab_data})

# the DataFrame Connector is associated with NtvType 'tab' in dicobj 
sr = field.to_obj(encode_format='obj', dicobj={'field': 'SeriesConnec'})
df = tab.to_obj  (encode_format='obj', dicobj={'tab': 'DataFrameConnec'})

# pandas dtype conform to Ntv type
print(sr, '\n')
print(df.dtypes)
df

In [ ]:
# the dataframe generated from NTV data from a first dataframe is identical to this first dataframe
df2 = Ntv.obj(df).to_obj(encode_format='obj', dicobj={'tab': 'DataFrameConnec'})
print('df2 is identical to df ? ', df2.equals(df))
